# 🌫️ Lahore AQI Predictor — Exploratory Data Analysis

This notebook performs a comprehensive EDA on the AQI and weather data collected for Lahore, Pakistan.

**Sections:**
1. Setup & Data Loading
2. Basic Data Overview
3. AQI Trend Analysis
4. Seasonal & Time Patterns
5. Pollutant Analysis
6. Feature Correlations
7. Outlier Detection
8. Feature Importance (Pre-Analysis)
9. Key Findings Summary

## 1. Setup & Data Loading

In [ ]:
import os
import sys
sys.path.append(os.path.abspath('../..'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import hopsworks
from dotenv import load_dotenv
from scipy import stats

load_dotenv()

# Plot styling
plt.rcParams['figure.figsize'] = (14, 5)
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
sns.set_palette('husl')

print('Libraries loaded ✓')

In [ ]:
# ── Load from Hopsworks Feature Store ────────────────────────────────────────
project = hopsworks.login(
    project=os.getenv('HOPSWORKS_PROJECT_NAME'),
    api_key_value=os.getenv('HOPSWORKS_API_KEY')
)
fs = project.get_feature_store()

aqi_fg = fs.get_feature_group(name='aqi_features', version=1)
try:
    df = aqi_fg.select_all().read()
except Exception:
    df = aqi_fg.select_all().read(read_options={'use_hive': True})

df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values('datetime').reset_index(drop=True)

print(f'Loaded {len(df):,} rows from Feature Store')
print(f'Date range: {df["datetime"].min().date()} → {df["datetime"].max().date()}')
print(f'Columns: {list(df.columns)}')

## 2. Basic Data Overview

In [ ]:
print('=== Shape ===')
print(f'Rows: {len(df):,}  |  Columns: {len(df.columns)}')

print('\n=== Data Types ===')
print(df.dtypes)

print('\n=== Missing Values ===')
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'Missing Count': missing, 'Missing %': missing_pct})
print(missing_df[missing_df['Missing Count'] > 0])

print('\n=== Descriptive Statistics ===')
df.describe().round(2)

In [ ]:
# Missing value heatmap
fig, ax = plt.subplots(figsize=(14, 4))
sns.heatmap(
    df.isnull().T,
    cbar=False, ax=ax,
    cmap=['white', 'red'],
    yticklabels=True
)
ax.set_title('Missing Value Map (red = missing)', fontsize=14, fontweight='bold')
ax.set_xlabel('Row Index')
plt.tight_layout()
plt.show()

## 3. AQI Trend Analysis

In [ ]:
# Full AQI time series
fig, ax = plt.subplots(figsize=(16, 5))
ax.plot(df['datetime'], df['aqi'], linewidth=0.8, color='#1A56DB', alpha=0.7, label='Hourly AQI')

# 7-day rolling average
rolling_7d = df.set_index('datetime')['aqi'].rolling('7D').mean()
ax.plot(rolling_7d.index, rolling_7d.values, linewidth=2, color='red', label='7-Day Rolling Avg')

# AQI threshold lines
thresholds = [(50, 'Good', '#00E400'), (100, 'Moderate', '#FFFF00'),
              (150, 'USG', '#FF7E00'), (200, 'Unhealthy', '#FF0000'),
              (300, 'Very Unhealthy', '#8F3F97')]
for val, label, color in thresholds:
    ax.axhline(val, linestyle='--', alpha=0.4, color=color, linewidth=1)
    ax.text(df['datetime'].iloc[-1], val + 3, label, fontsize=8, color=color)

ax.set_title('Lahore AQI Over Time', fontsize=15, fontweight='bold')
ax.set_ylabel('AQI')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# AQI Category Distribution
def aqi_category(val):
    if val <= 50:    return 'Good'
    elif val <= 100: return 'Moderate'
    elif val <= 150: return 'Unhealthy (Sensitive)'
    elif val <= 200: return 'Unhealthy'
    elif val <= 300: return 'Very Unhealthy'
    else:            return 'Hazardous'

cat_order  = ['Good', 'Moderate', 'Unhealthy (Sensitive)', 'Unhealthy', 'Very Unhealthy', 'Hazardous']
cat_colors = ['#00E400', '#FFFF00', '#FF7E00', '#FF0000', '#8F3F97', '#7E0023']

df['aqi_category'] = df['aqi'].apply(aqi_category)
cat_counts = df['aqi_category'].value_counts().reindex(cat_order, fill_value=0)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
bars = axes[0].bar(cat_counts.index, cat_counts.values, color=cat_colors, edgecolor='white')
axes[0].set_title('AQI Category Frequency', fontweight='bold')
axes[0].set_ylabel('Hours')
axes[0].tick_params(axis='x', rotation=30)
for bar, val in zip(bars, cat_counts.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                 f'{val:,}', ha='center', va='bottom', fontsize=9)

# Pie chart
non_zero = cat_counts[cat_counts > 0]
colors_nz = [cat_colors[i] for i, v in enumerate(cat_counts.values) if v > 0]
axes[1].pie(non_zero.values, labels=non_zero.index, colors=colors_nz,
            autopct='%1.1f%%', startangle=140)
axes[1].set_title('AQI Category Share', fontweight='bold')

plt.tight_layout()
plt.show()

print('\n📊 AQI Category Summary:')
print(cat_counts.to_frame('Hours').assign(Percentage=lambda x: (x['Hours']/len(df)*100).round(1)))

## 4. Seasonal & Time Patterns

In [ ]:
# Ensure time features
df['hour']    = df['datetime'].dt.hour
df['day']     = df['datetime'].dt.day
df['month']   = df['datetime'].dt.month
df['weekday'] = df['datetime'].dt.dayofweek
df['year']    = df['datetime'].dt.year

month_labels   = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
weekday_labels = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Hourly pattern
hourly_avg = df.groupby('hour')['aqi'].mean()
axes[0,0].plot(hourly_avg.index, hourly_avg.values, marker='o', color='#1A56DB', linewidth=2)
axes[0,0].fill_between(hourly_avg.index, hourly_avg.values, alpha=0.2, color='#1A56DB')
axes[0,0].set_title('Average AQI by Hour of Day', fontweight='bold')
axes[0,0].set_xlabel('Hour')
axes[0,0].set_ylabel('Avg AQI')
axes[0,0].set_xticks(range(0, 24, 2))
axes[0,0].grid(True, alpha=0.3)

# Monthly pattern
monthly_avg = df.groupby('month')['aqi'].mean()
bars = axes[0,1].bar(monthly_avg.index, monthly_avg.values, color='#E74C3C', alpha=0.8)
axes[0,1].set_title('Average AQI by Month', fontweight='bold')
axes[0,1].set_xlabel('Month')
axes[0,1].set_ylabel('Avg AQI')
axes[0,1].set_xticks(range(1, 13))
axes[0,1].set_xticklabels(month_labels)
axes[0,1].grid(True, alpha=0.3, axis='y')

# Weekday pattern
weekday_avg = df.groupby('weekday')['aqi'].mean()
axes[1,0].bar(weekday_avg.index, weekday_avg.values, color='#2ECC71', alpha=0.8)
axes[1,0].set_title('Average AQI by Day of Week', fontweight='bold')
axes[1,0].set_xlabel('Day')
axes[1,0].set_ylabel('Avg AQI')
axes[1,0].set_xticks(range(7))
axes[1,0].set_xticklabels(weekday_labels)
axes[1,0].grid(True, alpha=0.3, axis='y')

# Monthly boxplot
df_box = df[df['month'].between(1, 12)]
month_data = [df_box[df_box['month'] == m]['aqi'].values for m in range(1, 13)]
axes[1,1].boxplot(month_data, labels=month_labels, patch_artist=True,
                  boxprops=dict(facecolor='#3498DB', alpha=0.6))
axes[1,1].set_title('AQI Distribution by Month (Boxplot)', fontweight='bold')
axes[1,1].set_ylabel('AQI')
axes[1,1].tick_params(axis='x', rotation=30)
axes[1,1].grid(True, alpha=0.3, axis='y')

plt.suptitle('Temporal AQI Patterns — Lahore', fontsize=16, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Heatmap: Hour of day vs Month
pivot = df.groupby(['month', 'hour'])['aqi'].mean().unstack()

fig, ax = plt.subplots(figsize=(16, 6))
sns.heatmap(
    pivot, ax=ax,
    cmap='RdYlGn_r',
    annot=False,
    fmt='.0f',
    linewidths=0.3,
    cbar_kws={'label': 'Average AQI'}
)
ax.set_yticklabels(month_labels, rotation=0)
ax.set_xlabel('Hour of Day')
ax.set_ylabel('Month')
ax.set_title('AQI Heatmap: Month vs Hour of Day', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Pollutant Analysis

In [ ]:
pollutants   = ['pm10', 'pm2_5', 'co', 'no2', 'so2', 'o3', 'nh3']
pol_present  = [p for p in pollutants if p in df.columns]

fig, axes = plt.subplots(3, 3, figsize=(16, 12))
axes = axes.flatten()

for i, pol in enumerate(pol_present):
    axes[i].hist(df[pol].dropna(), bins=50, color=f'C{i}', alpha=0.8, edgecolor='white')
    axes[i].set_title(f'{pol.upper()} Distribution', fontweight='bold')
    axes[i].set_xlabel('Concentration')
    axes[i].set_ylabel('Frequency')
    mean_val = df[pol].mean()
    axes[i].axvline(mean_val, color='red', linestyle='--', linewidth=1.5, label=f'Mean: {mean_val:.1f}')
    axes[i].legend(fontsize=8)
    axes[i].grid(True, alpha=0.3)

# Hide unused subplots
for j in range(len(pol_present), len(axes)):
    axes[j].set_visible(False)

plt.suptitle('Pollutant Distributions', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Pollutant trends over time (last 30 days)
last_30 = df[df['datetime'] >= df['datetime'].max() - pd.Timedelta(days=30)]

fig, axes = plt.subplots(len(pol_present), 1, figsize=(16, 3 * len(pol_present)), sharex=True)

for i, pol in enumerate(pol_present):
    axes[i].plot(last_30['datetime'], last_30[pol], linewidth=1, color=f'C{i}', alpha=0.8)
    axes[i].set_ylabel(pol.upper(), fontsize=9)
    axes[i].grid(True, alpha=0.3)
    axes[i].fill_between(last_30['datetime'], last_30[pol], alpha=0.15, color=f'C{i}')

axes[0].set_title('Pollutant Levels — Last 30 Days', fontsize=14, fontweight='bold')
axes[-1].set_xlabel('Date')
plt.tight_layout()
plt.show()

In [ ]:
# Correlation of each pollutant with AQI
pol_corr = df[pol_present + ['aqi']].corr()['aqi'].drop('aqi').sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(10, 4))
colors = ['#2ECC71' if v >= 0 else '#E74C3C' for v in pol_corr.values]
ax.barh(pol_corr.index, pol_corr.values, color=colors, alpha=0.85, edgecolor='white')
ax.axvline(0, color='black', linewidth=0.8)
ax.set_xlabel('Pearson Correlation with AQI')
ax.set_title('Pollutant Correlation with AQI', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

for i, (val, name) in enumerate(zip(pol_corr.values, pol_corr.index)):
    ax.text(val + 0.01 if val >= 0 else val - 0.01, i,
            f'{val:.3f}', va='center', ha='left' if val >= 0 else 'right', fontsize=9)

plt.tight_layout()
plt.show()

print('\n🔍 Key Insight: Pollutants most correlated with AQI:')
print(pol_corr.round(3).to_string())

## 6. Feature Correlations

In [ ]:
# Full correlation heatmap
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# Exclude unix_time and index-like columns
exclude = ['unix_time', 'year']
numeric_cols = [c for c in numeric_cols if c not in exclude]

corr_matrix = df[numeric_cols].corr()

fig, ax = plt.subplots(figsize=(16, 13))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))  # Upper triangle mask
sns.heatmap(
    corr_matrix,
    mask=mask,
    annot=True, fmt='.2f',
    cmap='coolwarm', center=0,
    ax=ax,
    annot_kws={'size': 7},
    square=True,
    linewidths=0.5,
    cbar_kws={'shrink': 0.8}
)
ax.set_title('Full Feature Correlation Matrix (Lower Triangle)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Top correlations with AQI
aqi_corr = corr_matrix['aqi'].drop('aqi').abs().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(12, 5))
aqi_corr.head(15).plot(kind='bar', ax=ax, color='#1A56DB', alpha=0.85, edgecolor='white')
ax.set_title('Top 15 Features by Absolute Correlation with AQI', fontsize=13, fontweight='bold')
ax.set_ylabel('|Pearson r|')
ax.tick_params(axis='x', rotation=45)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print('\nTop 10 features correlated with AQI:')
print(aqi_corr.head(10).round(3).to_string())

In [ ]:
# Scatter plots: top 4 features vs AQI
top_features = aqi_corr.head(4).index.tolist()

fig, axes = plt.subplots(1, 4, figsize=(18, 4))
for i, feat in enumerate(top_features):
    axes[i].scatter(df[feat], df['aqi'], alpha=0.2, s=5, color=f'C{i}')
    # Regression line
    m, b, r, p, _ = stats.linregress(df[feat].dropna(), df.loc[df[feat].notna(), 'aqi'])
    x_line = np.linspace(df[feat].min(), df[feat].max(), 100)
    axes[i].plot(x_line, m * x_line + b, color='red', linewidth=2, label=f'r={r:.2f}')
    axes[i].set_xlabel(feat)
    axes[i].set_ylabel('AQI')
    axes[i].set_title(f'{feat} vs AQI', fontweight='bold')
    axes[i].legend(fontsize=9)
    axes[i].grid(True, alpha=0.3)

plt.suptitle('Top Feature Relationships with AQI', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 7. Outlier Detection

In [ ]:
# Z-score based outlier detection for AQI
df['aqi_zscore'] = np.abs(stats.zscore(df['aqi'].fillna(df['aqi'].mean())))
outliers = df[df['aqi_zscore'] > 3]

print(f'Total outliers (|z| > 3): {len(outliers):,} ({len(outliers)/len(df)*100:.2f}%)')
print(f'Outlier AQI range: {outliers["aqi"].min():.0f} – {outliers["aqi"].max():.0f}')

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Time series with outliers highlighted
axes[0].plot(df['datetime'], df['aqi'], linewidth=0.6, color='#1A56DB', alpha=0.5, label='AQI')
axes[0].scatter(outliers['datetime'], outliers['aqi'],
                color='red', s=15, zorder=5, label=f'Outliers ({len(outliers)})')
axes[0].set_title('AQI Time Series with Outliers', fontweight='bold')
axes[0].set_ylabel('AQI')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Box plot
axes[1].boxplot(df['aqi'].dropna(), vert=True, patch_artist=True,
                boxprops=dict(facecolor='#1A56DB', alpha=0.6),
                flierprops=dict(marker='o', color='red', alpha=0.5, markersize=3))
axes[1].set_title('AQI Box Plot', fontweight='bold')
axes[1].set_ylabel('AQI')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# IQR outlier check for all numeric features
outlier_summary = []
for col in pol_present + ['aqi', 'temp', 'humidity', 'pressure', 'wind_speed']:
    if col not in df.columns: continue
    Q1  = df[col].quantile(0.25)
    Q3  = df[col].quantile(0.75)
    IQR = Q3 - Q1
    n_out = ((df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)).sum()
    outlier_summary.append({
        'Feature': col,
        'Q1': round(Q1, 2), 'Q3': round(Q3, 2), 'IQR': round(IQR, 2),
        'Outliers (IQR)': n_out,
        'Outlier %': round(n_out / len(df) * 100, 2)
    })

pd.DataFrame(outlier_summary).set_index('Feature')

## 8. Feature Importance Pre-Analysis

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# Use available feature columns
FEATURE_COLS = [
    'temp', 'humidity', 'pressure', 'clouds', 'wind_speed', 'wind_deg',
    'pm10', 'pm2_5', 'co', 'no2', 'so2', 'o3', 'nh3',
    'hour', 'day', 'month', 'weekday',
    'aqi_lag_1h', 'aqi_lag_24h', 'aqi_change_rate'
]
feat_available = [c for c in FEATURE_COLS if c in df.columns]

df_clean = df[feat_available + ['aqi']].dropna()
X = df_clean[feat_available]
y = df_clean['aqi']

print(f'Training quick RF for feature importance on {len(X):,} samples...')
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X, y)

importance_df = pd.DataFrame({
    'Feature':    feat_available,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

fig, ax = plt.subplots(figsize=(12, 6))
colors = plt.cm.RdYlGn_r(np.linspace(0.1, 0.9, len(importance_df)))
ax.barh(importance_df['Feature'], importance_df['Importance'],
        color=colors, edgecolor='white')
ax.set_xlabel('Feature Importance (Mean Decrease in Impurity)')
ax.set_title('Random Forest Feature Importance for AQI Prediction',
             fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print('\nTop 10 Most Important Features:')
print(importance_df.head(10).to_string(index=False))

In [ ]:
# Lag feature effectiveness: autocorrelation of AQI
from pandas.plotting import autocorrelation_plot

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# ACF
aqi_series = df.set_index('datetime')['aqi'].dropna()
pd.plotting.autocorrelation_plot(aqi_series, ax=axes[0])
axes[0].set_xlim(0, 48)
axes[0].set_title('AQI Autocorrelation (first 48 lags)', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Lag scatter: AQI vs AQI 24h ago
if 'aqi_lag_24h' in df.columns:
    valid = df[['aqi', 'aqi_lag_24h']].dropna()
    axes[1].scatter(valid['aqi_lag_24h'], valid['aqi'], alpha=0.15, s=5, color='#1A56DB')
    m, b, r, _, _ = stats.linregress(valid['aqi_lag_24h'], valid['aqi'])
    x_line = np.linspace(valid['aqi_lag_24h'].min(), valid['aqi_lag_24h'].max(), 100)
    axes[1].plot(x_line, m*x_line+b, color='red', linewidth=2, label=f'r = {r:.3f}')
    axes[1].set_xlabel('AQI 24h ago')
    axes[1].set_ylabel('Current AQI')
    axes[1].set_title('AQI vs AQI Lagged 24 Hours', fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Key Findings Summary

In [ ]:
print('=' * 60)
print('  KEY FINDINGS — LAHORE AQI EDA')
print('=' * 60)

print(f"""
📊 DATASET
  Total records    : {len(df):,} hourly observations
  Date range       : {df['datetime'].min().date()} → {df['datetime'].max().date()}
  Missing values   : {df.isnull().sum().sum():,} total across all columns

🌫️ AQI STATISTICS
  Mean AQI         : {df['aqi'].mean():.1f}
  Median AQI       : {df['aqi'].median():.1f}
  Max AQI          : {df['aqi'].max():.1f}
  Min AQI          : {df['aqi'].min():.1f}
  Std Dev          : {df['aqi'].std():.1f}
  % Unhealthy+ (>150): {(df['aqi'] > 150).mean()*100:.1f}%

📅 TEMPORAL PATTERNS
  Worst month      : {month_labels[int(df.groupby('month')['aqi'].mean().idxmax())-1]}
  Best month       : {month_labels[int(df.groupby('month')['aqi'].mean().idxmin())-1]}
  Peak hour        : {int(df.groupby('hour')['aqi'].mean().idxmax())}:00
  Best hour        : {int(df.groupby('hour')['aqi'].mean().idxmin())}:00

🔬 TOP FEATURES (RF Importance)
""")
for _, row in importance_df.head(5).iterrows():
    print(f"  {row['Feature']:<20} {row['Importance']:.4f}")

print(f"""
⚠️ OUTLIERS
  Z-score > 3      : {len(outliers):,} records ({len(outliers)/len(df)*100:.2f}%)

✅ RECOMMENDATIONS FOR MODELLING
  1. Lag features (aqi_lag_1h, aqi_lag_24h) are strong predictors — keep them.
  2. PM2.5 and PM10 are the most correlated pollutants with AQI.
  3. Winter months (Nov–Jan) show highest AQI — include month as a feature.
  4. Standardise features before Ridge and LSTM training.
  5. Outliers are real pollution events — do NOT remove them.
""")
print('=' * 60)